## 2.4. 정규화와 피드 포워드 층

정규화란, 딥러닝 모델에서 입력이 일정한 분포를 갖도록 만들어 학습이 안정적이고 빠르게 이루어지도록 하는 방법이다. 과거에는 배치 정규화(Batch Normalization)이 주로 사용되었지만, 트랜스포머 아키텍처에서는 특징 차원에서 정규화를 수행하는 층 정규화(Layer Normalization)를 사용한다.
어텐션 연산이 입력 단어 사이의 관계를 계산해 토큰 임베딩을 조정하는 역할을 한다면 전체 입력문장을 이해하는 연산은 완전 연결 층(Fully Connected Layer)이다. 이 층은 입력과 출력의 차원이 같은 행렬 곱셈을 수행하고, 이후에 활성화 함수를 통과시켜 비선형성을 추가한다. 이러한 층을 피드 포워드 층(Feed Forward Layer)이라고 한다.

### 2.4.1. 층 정규화 이해하기

벡터 $x$를 정규화한 $norm_x$는 벡터 $x$에서 $x$의 평균을 빼고 $x$의 표준편차로 나눠 평균이 $0$이고 표준편차가 $1$인 분포로 만든다.
$$
\text{norm}_x = \frac{x - \mu}{\sigma}
$$

**배치 정규화(Batch Normalization)**

배치 정규화는 모델에 입력으로 들어가는 미니 배치 사이의 정규화를 수행한다. 자연어 처리에서는 배치 정규화를 사용하지 않는 이유는 자연어 처리에서 배치에 들어가는 입력의 길이가 다양하기 때문이다.

**층 정규화(Layer Normalization)**

층 정규화는 배치 정규화와 달리 각 토큰 임베딩의 평균과 표준편차를 구해 정규화를 수행한다. 이를 통해 입력의 분포를 안정화시킬 수 있다.
이려면 배치 정규화의 단점을 보완할 수 있다.

**층 정규화의 2가지 방식**

**사후 정규화(Post-LN)**
어텐션관 피드 포워드 층 이후에 층 정규화를 수행하는 방식이다.

**사전 정규화(Pre-LN)**
층 정규화를 수행한 후에 어텐션과 피드 포워드 층을 수행하는 방식이다.

In [2]:

# 예제 2.5 스케일 점곱 방식의 어텐션
from math import sqrt
import torch
import torch.nn as nn
import torch.nn.functional as F

# 입력 텐서 생성
input_text = "나는 최근 파리 여행을 다녀왔다"

input_text_list = input_text.split()
print("토큰화 결과: ", input_text_list)

# 토큰 -> 아이디 딕셔너리 생성
str2dix = {word: i for i, word in enumerate(input_text_list)}
print("토큰 -> 아이디 딕셔너리: ", str2dix)
# 아이디 -> 토큰 딕셔너리 생성
dix2str = {i: word for i, word in enumerate(input_text_list)}
print("아이디 -> 토큰 딕셔너리: ", dix2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2dix[word] for word in input_text_list]
print("토큰을 아이디로 변환: ", input_ids)

embedding_dim = 16 # 임베딩 차원
head_dim = 16 # 헤드 차원

embed_layer = nn.Embedding(len(str2dix), embedding_dim) # 임베딩 레이어 생성

input_embeddings = embed_layer(torch.tensor(input_ids)) # 토큰 아이디를 임베딩 벡터로 변환
input_embeddings = input_embeddings.unsqueeze(0) # 배치 차원 추가

# 쿼리, 키, 값을 게산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환
querys = weight_q(input_embeddings) # 쿼리
keys = weight_k(input_embeddings) # 키
values = weight_v(input_embeddings) # 값

def compute_attention(querys, keys, values, is_causal=False):

    dim_k = querys.size(-1)

    # 쿼리와 키를 곱하고 임베딩 차원 수의 제곱근으로 나눈다 (스케일)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    # 쿼리와 키를 곱해 계산한 스코어 값이 1이 되도록 소프트맥스를 취해 가중치를 구한다
    weights = F.softmax(scores, dim=-1) # 소프트맥스
    # 가중치와 값을 곱해 입력과 동일한 차원을 가진 출력을 만든다
    return weights @ values # 가중합

# 예제 2.6 스케일 점곱 어텐션을 사용한 멀티헤드 어텐션
class MultiHeadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, h_head, is_causal=False):
        super().__init__()
        self.n_head = h_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        
        B, T, C = querys.size()
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 1번째 선형 층 적용
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 2번째 선형 층 적용
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 3번째 선형 층 적용

        attention = compute_attention(querys, keys, values, self.is_causal) # h번의 스케일 점곱 어텐션 수행
        output = attention.transpose(1, 2).reshape(B, T, C) # 어텐션 결과를 연결
        output = self.concat_linear(output) # 마지막 선형층 적용
        return output
    
n_head = 4
mh_attention = MultiHeadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

토큰화 결과:  ['나는', '최근', '파리', '여행을', '다녀왔다']
토큰 -> 아이디 딕셔너리:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
아이디 -> 토큰 딕셔너리:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
토큰을 아이디로 변환:  [0, 1, 2, 3, 4]


torch.Size([1, 5, 16])

In [3]:
# 2.9 층 정규화 (사전 정규화)

import torch.nn as nn

norm = nn.LayerNorm(embedding_dim)
norm_x = norm(after_attention_embeddings)
print("norm_x shape: ", norm_x.shape)

print(norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data) # 실제로 평균과 표준편차 확인

norm_x shape:  torch.Size([1, 5, 16])
tensor([[7.4506e-09, 1.4901e-08, 4.0978e-08, 1.4901e-08, 3.7253e-08]]) tensor([[1.0325, 1.0326, 1.0326, 1.0326, 1.0326]])


### 2.4.2. 피드 포워드 층

**피드 포워드 층(Feed Forward Layer)**은 데이터의 특징을 학습하는 **완전 연결 층(Fully Connected Layer)**이다. 멀티 헤드 어텐션이 단어 사이의 관계를 파악하는 역할 이라면 피드 포워드 층은 전체 문장을 이해하는 역할을 한다.

**피드 포워드 신경망**
- 선형층
- 드롭아웃 층
- 정규화 층
- 활성화 함수

In [4]:
class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()

        self.linear1 = nn.Linear(d_model, dim_feedforward) # 1번째 선형 층
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 2번째 선형 층
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃 1
        self.dropout2 = nn.Dropout(dropout) # 드롭아웃 2
        self.activation = nn.GELU() # 활성화 함수
        self.norm = nn.LayerNorm(d_model) # 층 정규화

    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.activation(self.linear1(x))) # 2개의 선형 층과 활성화 함수
        x = self.dropout2(x) # 드롭아웃 2
        return x